In [1]:
import pandas as pd
plays_df = pd.read_csv('data/nfl-big-data-bowl-2021/plays.csv')
plays_df.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,...,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
0,2018090600,75,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,...,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
1,2018090600,146,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,...,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False
2,2018090600,168,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,...,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False
3,2018090600,190,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,...,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False
4,2018090600,256,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,...,0.0,10:59:00,11.0,NaN,NaN,I,0,0,-0.842272,False


In [2]:
plays_df.dtypes

gameId                      int64
playId                      int64
playDescription            object
quarter                     int64
down                        int64
yardsToGo                   int64
possessionTeam             object
playType                   object
yardlineSide               object
yardlineNumber              int64
offenseFormation           object
personnelO                 object
defendersInTheBox         float64
numberOfPassRushers       float64
personnelD                 object
typeDropback               object
preSnapVisitorScore       float64
preSnapHomeScore          float64
gameClock                  object
absoluteYardlineNumber    float64
penaltyCodes               object
penaltyJerseyNumbers       object
passResult                 object
offensePlayResult           int64
playResult                  int64
epa                       float64
isDefensivePI                bool
dtype: object

In [3]:
plays_df['play_uuid'] = plays_df['gameId'].astype(str) + '.' + plays_df['playId'].astype(str)
plays_df['play_uuid'].head()

0     2018090600.75
1    2018090600.146
2    2018090600.168
3    2018090600.190
4    2018090600.256
Name: play_uuid, dtype: object

In [4]:
third_down_df = plays_df.loc[(plays_df['down'] == 3)]
third_down_df.columns

Index(['gameId', 'playId', 'playDescription', 'quarter', 'down', 'yardsToGo',
       'possessionTeam', 'playType', 'yardlineSide', 'yardlineNumber',
       'offenseFormation', 'personnelO', 'defendersInTheBox',
       'numberOfPassRushers', 'personnelD', 'typeDropback',
       'preSnapVisitorScore', 'preSnapHomeScore', 'gameClock',
       'absoluteYardlineNumber', 'penaltyCodes', 'penaltyJerseyNumbers',
       'passResult', 'offensePlayResult', 'playResult', 'epa', 'isDefensivePI',
       'play_uuid'],
      dtype='object')

In [5]:
# prompt: load all CSV files with a name like "tracking_week_" into a tracking_df dataframe
import glob
tracking_df = pd.concat(map(pd.read_csv, glob.glob('data/nfl-big-data-bowl-2021/week*.csv')))

In [7]:
tracking_df['play_uuid'] = tracking_df['gameId'].astype(str) + '.' + tracking_df['playId'].astype(str)
tracking_df['play_uuid'].head()

0    2018090600.75
1    2018090600.75
2    2018090600.75
3    2018090600.75
4    2018090600.75
Name: play_uuid, dtype: object

In [8]:
tracking_df.loc[(tracking_df['play_uuid'].isin(third_down_df['play_uuid'])) & (tracking_df['frameId'] == 1)].head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route,play_uuid
2184,2018-09-07T01:10:34.599Z,54.30,29.58,0.18,0.11,0.02,285.29,254.04,NaN,310.0,Matt Ryan,2.0,QB,1,away,2018090600,190,left,NaN,2018090600.190
2185,2018-09-07T01:10:34.599Z,46.90,22.13,1.03,0.52,0.10,103.25,273.82,NaN,79848.0,Malcolm Jenkins,27.0,SS,1,home,2018090600,190,left,NaN,2018090600.190
2186,2018-09-07T01:10:34.599Z,50.41,21.58,0.01,0.01,0.00,259.09,154.62,NaN,2495454.0,Julio Jones,11.0,WR,1,away,2018090600,190,left,CORNER,2018090600.190
2187,2018-09-07T01:10:34.599Z,35.39,24.49,1.45,0.94,0.14,97.59,221.61,NaN,2495613.0,Corey Graham,24.0,FS,1,home,2018090600,190,left,NaN,2018090600.190
2188,2018-09-07T01:10:34.599Z,51.20,18.21,0.00,0.00,0.00,279.07,115.13,NaN,2533040.0,Mohamed Sanu,12.0,WR,1,away,2018090600,190,left,IN,2018090600.190
